In [14]:
import pandas as pd
import psycopg2 as pg
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from datetime import datetime, timedelta
import sys
sys.path.append('../../../')
from env import pg_conn

In [ ]:
today = datetime.today()
day = today.weekday()
print(day)
file_date = today.strftime("%y%m%d")
print(f"File date: {file_date}")


In [15]:
today = datetime.today()
day = today.weekday()
print(day)
file_date = today.strftime("%y%m%d")
print(f"File date: {file_date}")

visa_end = today.date() - timedelta(days=3) #Copy to start date
mca_end = today.date() - timedelta(days=2) #Copy to start date

if day == 0: #Monday
    visa_start = visa_end - timedelta(days=4)
    mca_start = mca_end - timedelta(days=4)
elif day == 3:
    visa_start = visa_end - timedelta(days=3)
    mca_start = mca_end - timedelta(days=3)

#overwrites below
visa_start = '2024-01-06 00:00:00'
mca_start = '2024-01-07 00:00:00' 
visa_end = '2024-01-06 00:00:00'
mca_end = '2024-01-07 00:00:00'

print(f"Mastercard/Amex Dates: {mca_start}-{mca_end}")
print(f"Visa Dates: {visa_start}-{visa_end}")

2
File date: 240124
2024-01-21
2024-01-22


In [16]:
def read_query(database, query):
    engine = create_engine(pg_conn+str(database))
    conn = engine.connect()
    transaction = conn.begin()
    try:
        # perform some database operations here
        df = pd.read_sql(query, conn)
        transaction.commit()
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also    
            print(df)
        return df
    except SQLAlchemyError as e:
        transaction.rollback()
        print(f"Error occurred: {e}")
    finally:
        conn.close()

### Getting all unmatched transactions from payment_transaction table
visa T-2 for all pending
Mastercard auth + settlement exists for all pending - settlement seen in transaction table
Amex same as mastercard - amex have different id for settlement 

SET MIN_DATE BELOW


In [17]:
harmonia_unmatched_query = f"""SELECT transaction_id, status, card_token, location_id, mid, spend_amount/100::NUMERIC(10,2) amount, transaction_date FROM payment_transaction pt
                            left join merchant_identifier mi on mi.identifier = pt.mid
where status = 'PENDING' and ((provider_slug = 'visa' and pt.created_at > '{visa_start}' and pt.created_at <= '{visa_end}' ) or provider_slug != 'visa')
and ((provider_slug != 'visa' and pt.created_at > '{mca_start}' and pt.created_at <= '{mca_end}' ) or provider_slug = 'visa')"""
harmonia_unmatched = read_query("harmonia", harmonia_unmatched_query)
harmonia_unmatched

                         transaction_id   status                   card_token  \
0                       000416476998792  PENDING  JtKytSItNhzc7YhorADYlgQWAE2   
1                    MCCGZ8UDU_20240117  PENDING  X7gM0on43OaBZryDmtSf5IAfN6p   
2                    MCCGZ8UDU_20240117  PENDING  X7gM0on43OaBZryDmtSf5IAfN6p   
3                       853537817340440  PENDING  BGCeTj1WaAuLGKLyf3gQAhljnsf   
4                    MDPPFN6DL_20240117  PENDING  WxE0JUCw2btEzLMw9a6dgNxYVxP   
5                    MDPPFN6DL_20240117  PENDING  WxE0JUCw2btEzLMw9a6dgNxYVxP   
6                       484017647917078  PENDING  ZAXdNJunEO85a5B0bjKiU1ZdvYm   
7                       484017647917078  PENDING  ZAXdNJunEO85a5B0bjKiU1ZdvYm   
8                    MNWGVYEDU_20240117  PENDING  KiuPor699dqn2TnPcUyFnP7zVbx   
9                    MNWGVYEDU_20240117  PENDING  KiuPor699dqn2TnPcUyFnP7zVbx   
10                      028131581301448  PENDING  4gyYkLDluFHDEZUou42XRiOq7lQ   
11                      7591

,transaction_id,status,card_token,location_id,mid,amount,transaction_date
0,000416476998792,PENDING,JtKytSItNhzc7YhorADYlgQWAE2,585,9448625699,6.55,2024-01-17 18:20:50+00:00
1,MCCGZ8UDU_20240117,PENDING,X7gM0on43OaBZryDmtSf5IAfN6p,500,7372311,7.00,2024-01-17 18:30:00+00:00
2,MCCGZ8UDU_20240117,PENDING,X7gM0on43OaBZryDmtSf5IAfN6p,500,7372311,7.00,2024-01-17 18:30:00+00:00
3,853537817340440,PENDING,BGCeTj1WaAuLGKLyf3gQAhljnsf,685,9422618272,10.60,2024-01-10 21:51:23+00:00
4,MDPPFN6DL_20240117,PENDING,WxE0JUCw2btEzLMw9a6dgNxYVxP,41,7373160,6.75,2024-01-17 17:59:00+00:00
...,...,...,...,...,...,...,...
16839,484017827205171,PENDING,FtLLkZNeX1B5rw9BWhatrE54DsW,663,1037014,12.84,2024-01-17 22:58:40+00:00
16840,484017827205171,PENDING,FtLLkZNeX1B5rw9BWhatrE54DsW,663,1037014,12.84,2024-01-17 22:58:40+00:00
16841,484019785713492,PENDING,3AAe3pqEgX6T4JBGJMzmYmQWRVS,456,7368277,9.00,2024-01-19 21:49:31+00:00
16842,484019785713492,PENDING,3AAe3pqEgX6T4JBGJMzmYmQWRVS,456,7368277,9.00,2024-01-19 21:49:31+00:00


In [27]:
# %%
harmonia_transaction_table_query = """SELECT transaction_id, feed_type  FROM transaction where merchant_slug = 'stonegate' and (feed_type = 'SETTLED' or payment_provider_slug = 'visa') and feed_type != 'MERCHANT'"""
harmonia_transaction_table = read_query("harmonia", harmonia_transaction_table_query)
harmonia_transaction_table.set_index('transaction_id', inplace=True)


OperationalError: (psycopg2.OperationalError) could not translate host name "uksouth-prod-57fu.postgres.database.azure.com" to address: nodename nor servname provided, or not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
ubiq_query = """SELECT ucs.payment_card_account_id, ucs.scheme_account_id, sa.card_number, pc.token, ucs.active_link FROM ubiquity_paymentcardschemeentry ucs
                    inner join scheme_schemeaccount sa on sa.id = ucs.scheme_account_id
                    inner join payment_card_paymentcardaccount pc on pc.id = ucs.payment_card_account_id
                    where sa.scheme_id = 393 and active_link = True
                     ;"""
ubiq = read_query("hermes", ubiq_query)
ubiq.set_index('token', inplace=True)

### Filter payment transaction table using transaction table and join card number into transaction file

joined = harmonia_unmatched.join(harmonia_transaction_table, on='transaction_id', how="inner")

joined_membership = joined.join(ubiq, on='card_token')
joined_membership

### Generate final file for output

fill in todays date for the file delivery
file dumped in repo make sure to remove before committing code back into repo

final_file = joined_membership.rename({"transaction_id": "transactionid",
                          "card_number": "membernumber",
                          "location_id": "retailerlocation_id",
                          "amount": "transactionamount",
                          "transaction_date": "transactiondate"
                           }, axis=1)[["transactionid",
                           "membernumber",
                           "retailerlocation_id",
                           "transactionamount",
                           "transactiondate"]]

final_file_nulls_dropped =  final_file[~final_file['membernumber'].isnull()].drop_duplicates()
final_file_nulls_dropped.to_csv(f'data/stonegate_unmatched/bink_unmatched_transactions_{file_date}.csv',index=False)


### Last bit of code is used to check for nulls where card number is not present

Use this to just check there aren't loads of examples - should be less than 50 odd

In [ ]:
null_file = final_file[final_file['membernumber'].isnull()].drop_duplicates()
null_file.to_csv(f'data/stonegate_unmatched/bink_unmatched_transactions_{file_date}_UNLINKED.csv',index=False)

NameError: name 'final_file' is not defined

In [ ]:
import os
os.getcwd()

'/Users/anandbhakta/python/snowdrops/notebooks/retailor/stonegate'

In [26]:
dig uksouth-prod-57fu.postgres.database.azure.com

SyntaxError: invalid decimal literal (725594931.py, line 1)